In [14]:
import re
import os
import sys
import cv2
import math
import time
import shutil

import pandas as pd
import numpy as np
import pydicom
from utils import *

import matplotlib.pyplot as plt
from tqdm import tqdm
from shutil import copy
from datetime import date
from pydicom.uid import UID, generate_uid
from multiprocessing import dummy as multiprocessing

from glob import glob 

# 1. video data 학습 위한 csv file 생성

In [35]:
dcm_root_paths = '/home/work/LUS/Dataset/processed_temporally_separated_dataset_dcm/*/*.dcm'
all_dcm_paths = glob(dcm_root_paths)
print(len(all_dcm_paths))

56


In [36]:
dcm_root_paths

'/home/work/LUS/Dataset/processed_temporally_separated_dataset_dcm/*/*.dcm'

In [37]:
TS_df = pd.read_excel('/home/work/LUS/Dataset/csv_files/Tempollary_Separated_Label.xlsx', index_col=False)

print(len(TS_df))

56


In [38]:
TS_df.head()

,Patient \nnumber,PatientID,Data_idx,StudyDate,Data_type,Point Zone,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score Leader,Lung Ultrasound Score Final
0,1,44937986,2,20240712,video,NaN,1_152,NaN,NaN,NaN,NaN,1,0,NaN
1,1,44937986,3,20240712,video,NaN,NaN,NaN,NaN,"1_49, 75_124","1_41, 79_119",0,3ce,NaN
2,1,44937986,4,20240712,video,NaN,NaN,NaN,NaN,"1_24, 44_100, 121_150","1_19, 51_96, 131_148",0,3ce,NaN
3,1,44937986,5,20240712,video,NaN,"38_45, 119_124","7_37, 80_114","47_59, 127_137",NaN,NaN,1,2,NaN
4,1,44937986,6,20240712,video,NaN,76_88,"41_61, 132_137","10_22, 66_72, 90_101, 147_151",NaN,NaN,1,2,NaN


In [39]:
print(v1_df.columns)

Index(['PatientID', 'Data_idx', 'StudyDate', 'Data_type', 'Point Zone',
       'A-line', 'B-line', 'Confluent B-line', 'Consolidation',
       'Pleural effusion', 'Lung sliding', 'Lung Ultrasound Score',
       'LUS score align', 'study_id'],
      dtype='object')


In [40]:
TS_df['study_id'] = [f"{row['PatientID']}_{row['Data_idx']}" for idx, row in TS_df.iterrows()]

TS_df.head()

,Patient \nnumber,PatientID,Data_idx,StudyDate,Data_type,Point Zone,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score Leader,Lung Ultrasound Score Final,study_id
0,1,44937986,2,20240712,video,NaN,1_152,NaN,NaN,NaN,NaN,1,0,NaN,44937986_2
1,1,44937986,3,20240712,video,NaN,NaN,NaN,NaN,"1_49, 75_124","1_41, 79_119",0,3ce,NaN,44937986_3
2,1,44937986,4,20240712,video,NaN,NaN,NaN,NaN,"1_24, 44_100, 121_150","1_19, 51_96, 131_148",0,3ce,NaN,44937986_4
3,1,44937986,5,20240712,video,NaN,"38_45, 119_124","7_37, 80_114","47_59, 127_137",NaN,NaN,1,2,NaN,44937986_5
4,1,44937986,6,20240712,video,NaN,76_88,"41_61, 132_137","10_22, 66_72, 90_101, 147_151",NaN,NaN,1,2,NaN,44937986_6


In [41]:
def create_path(row):
    root_path = "/home/work/LUS/Dataset/processed_temporally_separated_dataset_dcm"
    patient_id = int(row['PatientID'])
    data_idx = f"{row['Data_idx']:05d}"
    return f"{root_path}/{patient_id}/{data_idx}.dcm"

In [42]:
TS_df['dcm_path'] = TS_df.apply(create_path, axis=1)

print(TS_df.iloc[0]['dcm_path'])

/home/work/LUS/Dataset/processed_temporally_separated_dataset_dcm/44937986/00002.dcm


# 2. 모든 영상의 video를 dcm to avi 형식으로 변환

In [43]:
def dicom_2_avi(video_path, destinationFolder, folder_name, file_name):

    # fileName = dicom_path.split('/')[-1]

    dataset = pydicom.dcmread(video_path, force=True)
    if dataset.PixelData:

        test_arr = dataset.pixel_array
        if len(test_arr.shape) == 4:  # 동영상인 case만 처리

            # get information to crop each frame
            xmin = dataset[(0x018, 0x6011)].value[0][(0x018, 0x6018)].value
            ymin = dataset[(0x018, 0x6011)].value[0][(0x018, 0x601a)].value
            xmax = dataset[(0x018, 0x6011)].value[0][(0x018, 0x601c)].value
            ymax = dataset[(0x018, 0x6011)].value[0][(0x018, 0x601e)].value

            frames, height, width, channels = test_arr.shape
            cropped_height = ymax-ymin 
            cropped_width = xmax-xmin
            fps = 30

            try:
                fps = dataset[(0x18, 0x40)].value
            except:
                print("couldn't find frame rate, default to 30")

            fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
            video_filename = os.path.join(destinationFolder, folder_name, file_name.replace('.dcm', '') + '.avi')
            out = cv2.VideoWriter(video_filename, fourcc, fps, (cropped_width, cropped_height))
            
            for i in range(frames):

                outputA = test_arr[i, :, :, 0]

                # crop image
                cropped_frame = outputA[ymin:ymax, xmin:xmax]
                
                # if you want resize frame, use it
                # output = cv2.resize(smallOutput, cropSize, interpolation=cv2.INTER_CUBIC)
                
                finaloutput = cv2.merge([cropped_frame, cropped_frame, cropped_frame])
                out.write(finaloutput)
            
            out.release()
        
        else:
            print(f'{file_name} is not a video file.')
            return 0
    else:
        print(f'{file_name} is empty.')
        return 0

# SNUH_MICU crop point
- (x, y, w, h) = (160 109 989 729)
- Top Left: (160, 109)
- Top Right: (1149, 109)
- Bottom Left: (160, 838)
- Bottom Right: (1149, 838)

In [33]:
'''# dcm header 정보를 이용해서 crop하는 method가 아닌 cv method 적용한 version
def dicom_2_avi_v2(video_path, destinationFolder, folder_name, file_name):

    # fileName = dicom_path.split('/')[-1]

    dataset = pydicom.dcmread(video_path, force=True)
    if dataset.PixelData:

        test_arr = dataset.pixel_array
        if len(test_arr.shape) == 4:  # 동영상인 case만 처리


            frames, height, width, channels = test_arr.shape
                    
            fps = 30

            fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
            video_filename = os.path.join(destinationFolder, folder_name, file_name.replace('.dcm', '') + '.avi')
            
            # snuh 기준
            x, y, w, h = 160, 109, 989, 729
            
            out = cv2.VideoWriter(video_filename, fourcc, fps, (w, h))
            
            for i in range(frames):
                
                org_img = test_arr[i, :, :, 0]

                # crop image
                cropped_img = org_img[y:y+h, x:x+w]
                cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_GRAY2BGR)
                
                out.write(cropped_img)
            
            out.release()
        
        else:
            print(f'{file_name} is not a video file.')
            return 0
    else:
        print(f'{file_name} is empty.')
        return 0'''

In [44]:
destination_folder = '/home/work/LUS/Dataset/processed_temporally_separated_dataset_avi'
# destination_folder = '/home/work/LUS/Dataset/ex'

for index, row in tqdm(TS_df.iterrows(), total=len(TS_df)):
    dcm_path = row['dcm_path']
    
    # 파일 이름과 폴더 이름 추출
    file_name = os.path.basename(dcm_path)
    folder_name = os.path.basename(os.path.dirname(dcm_path))
    
    # 대상 폴더 생성
    os.makedirs(os.path.join(destination_folder, folder_name), exist_ok=True)
    
    # DICOM을 AVI로 변환
    dicom_2_avi(dcm_path, destination_folder, folder_name, file_name)
#     dicom_2_avi_v2(dcm_path, destination_folder, folder_name, file_name)

print("변환 작업이 완료되었습니다.")

100% 56/56 [02:13<00:00,  2.39s/it]

변환 작업이 완료되었습니다.


In [35]:
a = [i for i in TS_df['PatientID']]
print(set(a))

{44937986, 59202567, 61398238, 14184938, 80306634, 60916943, 37292786, 54258868, 50647226, 61308381, 52426014}


In [40]:
# Replace 'processed_dataset_dcm' with 'processed_dataset_avi' and change the extension to '.avi'
def convert_to_avi_path(dcm_path):
    return dcm_path.replace('processed_dataset_dcm', 'processed_dataset_avi').replace('.dcm', '.avi')

# Apply the function to create the 'avi_path' column
TS_df['avi_path'] = TS_df['dcm_path'].apply(convert_to_avi_path)

print(TS_df['avi_path'].iloc[0])

/home/work/LUS/Dataset/processed_dataset_avi/44937986/00002.avi
/home/work/LUS/Dataset/processed_dataset_avi/44937986/00002.avi
